## LGBM, 랜덤포레스트 앙상블

In [2]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
import chardet
import pandas as pd
import numpy as npb
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

In [3]:
train_df = pd.read_csv("C:/Users/shin jaeik/Documents/sas_data_project/DATA/TRAIN_DATA.csv",encoding= 'euc-kr')
print(train_df.head())
print(train_df.info())

   DATA_YM  AREA_ID AREA_NM  DIST_CD DIST_NM  TOTAL_BIDG  FAC_NEIGH_1  \
0   202303     9626  중앙로역_4    27110      중구       538.0        266.0   
1   202303    10350    황리단길    47130     경주시       765.0         72.0   
2   202212     9547    농소1동    31200      북구       343.0         74.0   
3   202312     9995  심석고등학교    41360    남양주시       129.0          4.0   
4   202301     9248     고덕역    11740     강동구        34.0          2.0   

   FAC_NEIGH_2  FAC_CULT_MTG  FAC_RELG  FAC_RETAIL  FAC_MEDI  FAC_YOSE  \
0         88.0           6.0       3.0         2.0       3.0       0.0   
1        174.0           1.0       0.0         2.0       0.0       1.0   
2        161.0           1.0       3.0         0.0       2.0       6.0   
3         14.0           0.0       1.0         0.0       0.0       0.0   
4         14.0           0.0       0.0         1.0       0.0       1.0   

   FAC_TRAIN  FAC_SPORT  FAC_STAY  FAC_LEISURE  TOTAL_GAS  CMRC_GAS  \
0        0.0        0.0      16.0          3.

In [4]:
# 결측치 행 제거(3개),지역명 및 상권몇 제거
train_df = train_df.dropna()
train_df = train_df.drop(['DIST_NM','AREA_NM'], axis = 1)
train_df.head()
train_df.info()


# 시계열 데이터 변환
train_df['DATA_YM'] = pd.to_datetime(train_df['DATA_YM'], format='%Y%m')
# 연도, 월 분리, 원래 시간 데이터 제거
train_df['YEAR'] = train_df['DATA_YM'].dt.year
train_df['MONTH'] = train_df['DATA_YM'].dt.month
train_df = train_df.drop(['DATA_YM'], axis = 1)

train_df.head()
train_df.info()

#년, 월 칼럼 앞으로 뺴기
cols = ['YEAR','MONTH'] + [col for col in train_df.columns if col not in ['YEAR','MONTH']]
train_df = train_df[cols]
train_df.head()
train_df.info()

#xgbbooast는 트리결정모델이라 크기를 상관없으니 년, 월, 상권코드는 냅두고, 연속형 범주들 로그로 치환.

# 원본 데이터 복사
train_df_processed = train_df.copy()

# 로그 변환 제외할 컬럼 4개
exclude_cols = ['YEAR', 'MONTH', 'DIST_CD', 'AREA_ID']

# 변환 대상: 제외할 컬럼을 빼고 나머지 수치형 변수들
log_features = [col for col in train_df_processed.columns if col not in exclude_cols]

# 로그 변환 (log1p 사용)
for col in log_features:
    train_df_processed[col] = np.log1p(train_df_processed[col])

# 변환된 데이터 확인
train_df_processed.head()

df = train_df_processed.copy()




#데이터 전체 상관계수
correlation_matrix = df.corr()



#x,y분류
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

print(X)
print(y)


<class 'pandas.core.frame.DataFrame'>
Index: 26096 entries, 0 to 26098
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DATA_YM       26096 non-null  int64  
 1   AREA_ID       26096 non-null  int64  
 2   DIST_CD       26096 non-null  int64  
 3   TOTAL_BIDG    26096 non-null  float64
 4   FAC_NEIGH_1   26096 non-null  float64
 5   FAC_NEIGH_2   26096 non-null  float64
 6   FAC_CULT_MTG  26096 non-null  float64
 7   FAC_RELG      26096 non-null  float64
 8   FAC_RETAIL    26096 non-null  float64
 9   FAC_MEDI      26096 non-null  float64
 10  FAC_YOSE      26096 non-null  float64
 11  FAC_TRAIN     26096 non-null  float64
 12  FAC_SPORT     26096 non-null  float64
 13  FAC_STAY      26096 non-null  float64
 14  FAC_LEISURE   26096 non-null  float64
 15  TOTAL_GAS     26096 non-null  int64  
 16  CMRC_GAS      26096 non-null  int64  
 17  TOTAL_ELEC    26096 non-null  float64
dtypes: float64(13), int64(5)
memory

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
import optuna

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
def objective(trial):
    # LGBM 파라미터
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('lgbm_max_depth', 3, 12),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 20, 150),
    }
    
    # RF 파라미터
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('rf_max_depth', 3, 12),
    }

    lgbm = LGBMRegressor(**lgbm_params)
    rf = RandomForestRegressor(**rf_params)

    # 앙상블 (가중 평균 X, 단순 평균 Voting)
    model = VotingRegressor(estimators=[
        ('lgbm', lgbm),
        ('rf', rf)
    ])

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, preds, squared=False)
    return rmse

In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print("Best trial:")
print(study.best_trial)

[I 2025-04-11 15:26:22,529] A new study created in memory with name: no-name-60a1fac9-cc27-4ab4-8d83-b3f6622e8012


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006958 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1923
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 18
[LightGBM] [Info] Start training from score 6.500556


[I 2025-04-11 15:26:37,129] Trial 0 finished with value: 0.4517440387781999 and parameters: {'lgbm_n_estimators': 560, 'lgbm_learning_rate': 0.17819652372492803, 'lgbm_max_depth': 11, 'lgbm_num_leaves': 33, 'rf_n_estimators': 146, 'rf_max_depth': 5}. Best is trial 0 with value: 0.4517440387781999.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002623 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1923
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 18
[LightGBM] [Info] Start training from score 6.500556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-04-11 15:27:24,813] Trial 1 finished with value: 0.4398821662438034 and parameters: {'lgbm_n_estimators': 688, 'lgbm_learning_rate': 0.09343455834328855, 'lgbm_max_depth': 4, 'lgbm_num_leaves': 149, 'rf_n_estimators': 309, 'rf_max_depth': 9}. Best is trial 1 with value: 0.4398821662438034.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1923
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 18
[LightGBM] [Info] Start training from score 6.500556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-04-11 15:34:07,818] Trial 8 finished with value: 0.36037014780297766 and parameters: {'lgbm_n_estimators': 937, 'lgbm_learning_rate': 0.17349983148420636, 'lgbm_max_depth': 5, 'lgbm_num_leaves': 40, 'rf_n_estimators': 149, 'rf_max_depth': 10}. Best is trial 8 with value: 0.36037014780297766.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1923
[LightGBM] [Info] Number of data points in the train set: 20876, number of used features: 18
[LightGBM] [Info] Start training from score 6.500556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-04-11 15:36:38,634] Trial 9 finished with value: 0.36004926684943717 and parameters: {'lgbm_n_estimators': 687, 'lgbm_learning_rate': 0.27717757652332126, 'lgbm_max_depth': 7, 'lgbm_num_leaves': 76, 'rf_n_estimators': 981, 'rf_max_depth': 9}. Best is trial 9 with value: 0.36004926684943717.


Best trial:
FrozenTrial(number=9, state=1, values=[0.36004926684943717], datetime_start=datetime.datetime(2025, 4, 11, 15, 34, 7, 819592), datetime_complete=datetime.datetime(2025, 4, 11, 15, 36, 38, 632842), params={'lgbm_n_estimators': 687, 'lgbm_learning_rate': 0.27717757652332126, 'lgbm_max_depth': 7, 'lgbm_num_leaves': 76, 'rf_n_estimators': 981, 'rf_max_depth': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lgbm_n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'lgbm_learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'lgbm_max_depth': IntDistribution(high=12, log=False, low=3, step=1), 'lgbm_num_leaves': IntDistribution(high=150, log=False, low=20, step=1), 'rf_n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'rf_max_depth': IntDistribution(high=12, log=False, low=3, step=1)}, trial_id=9, value=None)


In [9]:
best_params = study.best_trial.params

lgbm_best = LGBMRegressor(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    max_depth=best_params['lgbm_max_depth'],
    num_leaves=best_params['lgbm_num_leaves']
)

rf_best = RandomForestRegressor(
    n_estimators=best_params['rf_n_estimators'],
    max_depth=best_params['rf_max_depth']
)

final_model = VotingRegressor(estimators=[
    ('lgbm', lgbm_best),
    ('rf', rf_best)
])

final_model.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.161233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1928
[LightGBM] [Info] Number of data points in the train set: 26096, number of used features: 18
[LightGBM] [Info] Start training from score 6.501273
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

VotingRegressor(estimators=[('lgbm',
                             LGBMRegressor(learning_rate=0.27717757652332126,
                                           max_depth=7, n_estimators=687,
                                           num_leaves=76)),
                            ('rf',
                             RandomForestRegressor(max_depth=9,
                                                   n_estimators=981))])

In [10]:
y_pred = final_model.predict(X_valid)

rmse = mean_squared_error(y_valid, y_pred, squared=False)
print(f"lgbm 포레스트 앙상블 모델 RMSE: {rmse:.4f}")

lgbm 포레스트 앙상블 모델 RMSE: 0.2491
